# Prerequisites

[AutoGluon](https://github.com/awslabs/autogluon) automates machine learning tasks enabling you to easily achieve strong predictive performance in your applications. With just a few lines of code, you can train and deploy high-accuracy deep learning models on tabular, image, and text data.
This example shows how to use AutoGluon-Tabular with Amazon SageMaker by applying [pre-built deep learning containers](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#autogluon-training-containers).

In [ ]:
import sagemaker
import pandas as pd
import numpy as np
from ag_model import AutoGluonTraining, AutoGluonInferenceModel, AutoGluonTabularPredictor
from sagemaker import utils
from sagemaker.serializers import CSVSerializer
import os

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session._region_name

bucket = sagemaker_session.default_bucket()
s3_prefix = f"autogluon_sm/{utils.sagemaker_timestamp()}"
output_path = f"s3://{bucket}/{s3_prefix}/output/"

### Get the data
We'll be using the [Adult Census dataset](https://archive.ics.uci.edu/ml/datasets/adult) for this exercise. 
This data was extracted from the [1994 Census bureau database](http://www.census.gov/en.html) by Ronny Kohavi and Barry Becker (Data Mining and Visualization, Silicon Graphics), with the task being to predict if an individual person makes over 50K a year. 

In [ ]:
!mkdir -p data

In [ ]:
columns = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "class",
]

In [ ]:
df_train = pd.read_csv(
    "s3://sagemaker-sample-files/datasets/tabular/uci_adult/adult.data", header=None
)
df_train.columns = columns
df_train.to_csv("data/train.csv")

In [ ]:
df_test = pd.read_csv(
    "s3://sagemaker-sample-files/datasets/tabular/uci_adult/adult.test", header=None, skiprows=1
)
df_test.columns = columns
df_test["class"] = df_test["class"].map(
    {
        " <=50K.": " <=50K",
        " >50K.": " >50K",
    }
)
df_test.to_csv("data/test.csv")

# Training

Users can create their own training/inference scripts using [SageMaker Python SDK examples](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script).
The scripts we created allow to pass AutoGluon configuration as a YAML file (located in `data/config` directory).

We are using [official](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#autogluon-training-containers) 0.3.1 AutoGluon Deep Learning Container images with custom training scripts (see `scripts/` directory).

In [ ]:
ag = AutoGluonTraining(
    role=role,
    entry_point="scripts/tabular_train.py",
    region=region,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    base_job_name="autogluon-tabular-train",
)

Upload the data to s3

In [ ]:
s3_prefix = f"autogluon_sm/{utils.sagemaker_timestamp()}"
train_input = ag.sagemaker_session.upload_data(
    path=os.path.join("data", "train.csv"), key_prefix=s3_prefix
)
eval_input = ag.sagemaker_session.upload_data(
    path=os.path.join("data", "test.csv"), key_prefix=s3_prefix
)
config_input = ag.sagemaker_session.upload_data(
    path=os.path.join("config", "config-med.yaml"), key_prefix=s3_prefix
)

### Fit The Model
For local training set train_instance_type to local.

For non-local training the recommended instance type is `ml.m5.2xlarge`.

In [ ]:
job_name = utils.unique_name_from_base("test-autogluon-image")
ag.fit({"config": config_input, "train": train_input, "test": eval_input}, job_name=job_name)

### Model export

AutoGluon models are portable: everything needed to deploy a trained model is in the tarball created by SageMaker.

The artifact can be used locally, on EC2/ECS/EKS or served via SageMaker Inference.

In [ ]:
!aws s3 cp {ag.model_data} .

In [ ]:
!ls -alF model.tar.gz

# Endpoint Deployment

Upload the model we trained earlier

In [ ]:
endpoint_name = sagemaker.utils.unique_name_from_base("sagemaker-autogluon-serving-trained-model")

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

Deploy remote or local endpoint

In [ ]:
model = AutoGluonInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    source_dir="scripts",
    entry_point="tabular_serve.py",
)

In [ ]:
instance_type = "ml.m5.2xlarge"
# instance_type = 'local'

In [ ]:
predictor = model.deploy(
    initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type
)

### Predict on unlabeled test data

In [ ]:
df = pd.read_csv("data/test.csv")
data = df.drop(columns="class")[:100].values

In [ ]:
preds = predictor.predict(data)

In [ ]:
p = pd.DataFrame({"preds": pd.DataFrame(preds)[0], "actual": df["class"][: len(preds)]})
p.head()

In [ ]:
print(f"{(p.preds==p.actual).astype(int).sum()}/{len(p)} are correct")

### Cleanup Endpoint

In [ ]:
predictor.delete_endpoint()

# Batch Transform

In [ ]:
endpoint_name = sagemaker.utils.unique_name_from_base(
    "sagemaker-autogluon-batch_transform-trained-model"
)

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [ ]:
model = AutoGluonInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    entry_point="tabular_serve-batch.py",
    source_dir="scripts",
    predictor_cls=AutoGluonTabularPredictor,
)

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    strategy="MultiRecord",
    max_payload=6,
    max_concurrent_transforms=1,
    output_path=output_path,
    accept="application/json",
    assemble_with="Line",
)

Prepare data for batch transform

In [ ]:
pd.read_csv(f"data/test.csv")[:100].to_csv("data/test_no_header.csv", header=False, index=False)

Upload data to sagemaker session

In [ ]:
test_input = transformer.sagemaker_session.upload_data(
    path=os.path.join("data", "test_no_header.csv"), key_prefix=s3_prefix
)

In [ ]:
transformer.transform(
    test_input,
    input_filter="$[:14]",  # filter-out target variable
    split_type="Line",
    content_type="text/csv",
    output_filter="$['class']",  # keep only prediction class in the output
)

transformer.wait()

Download batch transform outputs

In [ ]:
!aws s3 cp {transformer.output_path[:-1]}/test_no_header.csv.out .

In [ ]:
p = pd.concat(
    [
        pd.read_json("test_no_header.csv.out", orient="index")
        .sort_index()
        .rename(columns={0: "preds"}),
        pd.read_csv("data/test.csv")[["class"]].iloc[:100].rename(columns={"class": "actual"}),
    ],
    axis=1,
)
p.head()

In [ ]:
print(f"{(p.preds==p.actual).astype(int).sum()}/{len(p)} are correct")